In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nest

from scripts import visualizations, initializations, optimization, experiments, model


              -- N E S T --
  Copyright (C) 2004 The NEST Initiative

 Version: 3.6.0
 Built: Sep 28 2023 11:53:34

 This program is provided AS IS and comes with
 NO WARRANTY. See the file LICENSE for details.

 Problems or suggestions?
   Visit https://www.nest-simulator.org

 Type 'nest.help()' to find out more about NEST.



In [2]:
weights = np.load('/hpc/mzhu843/modelling/nest/results/optimization/optimised_weights.npy')

model1 = model.NeuronalNetwork(weights)
model1.simulate()
print(len(model1.voltage_traces[2]))






Feb 12 16:24:06 NodeManager::prepare_nodes [Info]: 
    Preparing 281 nodes for simulation.

Feb 12 16:24:06 SimulationManager::start_updating_ [Info]: 
    Number of local nodes: 281
    Simulation time (ms): 17988
    Number of OpenMP threads: 1
    Not using MPI

Feb 12 16:24:08 SimulationManager::run [Info]: 
    Simulation finished.
17987
